# CSET419 – Introduction to Generative AI
## Lab 2 – Experiment 2
### GAN for Fashion-MNIST Image Generation

**Objective:**  
To train a Generative Adversarial Network (GAN) to generate synthetic Fashion-MNIST clothing images and analyze the generated outputs.


In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [3]:
epochs = 50
batch_size = 64
noise_dim = 100
learning_rate = 0.0002
save_interval = 5


In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = datasets.FashionMNIST(
    root='./data',
    train=True,
    transform=transform,
    download=True
)

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


100%|██████████| 26.4M/26.4M [00:02<00:00, 12.0MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 205kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.80MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 25.9MB/s]


In [5]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        img = img.view(img.size(0), -1)
        return self.model(img)


In [6]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(noise_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 28*28),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        return img.view(z.size(0), 1, 28, 28)


In [7]:
generator = Generator().to(device)
discriminator = Discriminator().to(device)


In [8]:
adversarial_loss = nn.BCELoss()

optimizer_G = optim.Adam(generator.parameters(), lr=learning_rate)
optimizer_D = optim.Adam(discriminator.parameters(), lr=learning_rate)


In [9]:
os.makedirs("generated_samples", exist_ok=True)

for epoch in range(1, epochs+1):
    for real_imgs, _ in dataloader:

        real_imgs = real_imgs.to(device)
        batch_size_curr = real_imgs.size(0)

        # Train Discriminator
        optimizer_D.zero_grad()

        real_labels = torch.ones(batch_size_curr, 1, device=device)
        fake_labels = torch.zeros(batch_size_curr, 1, device=device)

        real_loss = adversarial_loss(discriminator(real_imgs), real_labels)

        noise = torch.randn(batch_size_curr, noise_dim, device=device)
        fake_imgs = generator(noise)
        fake_loss = adversarial_loss(discriminator(fake_imgs.detach()), fake_labels)

        d_loss = real_loss + fake_loss
        d_loss.backward()
        optimizer_D.step()

        # Train Generator
        optimizer_G.zero_grad()
        g_loss = adversarial_loss(discriminator(fake_imgs), real_labels)
        g_loss.backward()
        optimizer_G.step()

    print(f"Epoch {epoch}/{epochs} | D_loss: {d_loss.item():.4f} | G_loss: {g_loss.item():.4f}")

    if epoch % save_interval == 0:
        save_image(fake_imgs[:25], f"generated_samples/epoch_{epoch:02d}.png",
                   nrow=5, normalize=True)


Epoch 1/50 | D_loss: 0.9319 | G_loss: 2.9195
Epoch 2/50 | D_loss: 0.1069 | G_loss: 4.0030
Epoch 3/50 | D_loss: 0.0636 | G_loss: 5.0266
Epoch 4/50 | D_loss: 0.3880 | G_loss: 3.9242
Epoch 5/50 | D_loss: 0.3484 | G_loss: 3.4231
Epoch 6/50 | D_loss: 0.4519 | G_loss: 3.7530
Epoch 7/50 | D_loss: 0.4149 | G_loss: 1.7600
Epoch 8/50 | D_loss: 0.4775 | G_loss: 3.8177
Epoch 9/50 | D_loss: 0.5569 | G_loss: 2.9112
Epoch 10/50 | D_loss: 0.7102 | G_loss: 2.1881
Epoch 11/50 | D_loss: 0.7838 | G_loss: 1.5023
Epoch 12/50 | D_loss: 1.2067 | G_loss: 2.3191
Epoch 13/50 | D_loss: 0.5225 | G_loss: 2.5338
Epoch 14/50 | D_loss: 0.7078 | G_loss: 1.6437
Epoch 15/50 | D_loss: 0.9040 | G_loss: 1.8642
Epoch 16/50 | D_loss: 1.0668 | G_loss: 1.5264
Epoch 17/50 | D_loss: 0.8864 | G_loss: 1.8500
Epoch 18/50 | D_loss: 0.8693 | G_loss: 1.6333
Epoch 19/50 | D_loss: 1.1876 | G_loss: 1.6648
Epoch 20/50 | D_loss: 0.9450 | G_loss: 1.2078
Epoch 21/50 | D_loss: 0.8497 | G_loss: 1.3823
Epoch 22/50 | D_loss: 0.9162 | G_loss: 1.53

In [10]:
os.makedirs("final_generated_images", exist_ok=True)

generator.eval()
with torch.no_grad():
    noise = torch.randn(100, noise_dim, device=device)
    fake_images = generator(noise)
    save_image(fake_images,
               "final_generated_images/final_100_fashion_images.png",
               nrow=10, normalize=True)


## Results and Observations

- The GAN successfully learned Fashion-MNIST image patterns.
- Generated images resemble clothing items such as shirts, shoes, and bags.
- Compared to MNIST, Fashion-MNIST generation is more complex and less sharp.


In [11]:
class FashionClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        return self.model(x)


In [12]:
classifier = FashionClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer_C = optim.Adam(classifier.parameters(), lr=0.001)

classifier.train()
for epoch in range(3):  # small training
    for imgs, labels in dataloader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer_C.zero_grad()
        outputs = classifier(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_C.step()


In [13]:
classifier.eval()

with torch.no_grad():
    outputs = classifier(fake_images)
    predicted_labels = torch.argmax(outputs, dim=1)

labels, counts = torch.unique(predicted_labels, return_counts=True)

print("Label Distribution of Generated Fashion Images:")
for l, c in zip(labels.cpu().numpy(), counts.cpu().numpy()):
    print(f"Class {l}: {c} images")


Label Distribution of Generated Fashion Images:
Class 0: 7 images
Class 1: 9 images
Class 2: 6 images
Class 3: 14 images
Class 4: 7 images
Class 5: 20 images
Class 6: 7 images
Class 7: 10 images
Class 8: 4 images
Class 9: 16 images


## Results and Observations

- A simple Fashion-MNIST classifier was used to evaluate GAN outputs.
- The classifier predicted meaningful clothing categories for generated images.
- Compared to MNIST, Fashion-MNIST generation is more complex and visually challenging.
